In [2]:
# Manually set the start date and end date
start_date = datetime(2021, 2, 1)
end_date = datetime(2021, 2, 5)

In [3]:
# Initialize Redis client
redis_client = redis.StrictRedis(host='localhost', port=6379, db=0)

# Generate the list of dates in the format needed for the URL
def generate_date_list(start_date, end_date):
    return [(start_date + timedelta(days=x)).strftime('%b%d.%Y').lower() for x in range((end_date - start_date).days + 1)]

# Format date to "Day, Number Month"
def format_date_for_table(date_str):
    date_obj = datetime.strptime(date_str, '%b%d.%Y')
    return date_obj.strftime('%A, %d %B')

# Setup Chrome options for Selenium
chrome_options = Options()
chrome_options.binary_location = chrome_path
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Initialize undetected ChromeDriver
driver = uc.Chrome(options=chrome_options, use_subprocess=True)

# Prepare to store all collected data
all_events = []

# Function to fetch data for the given date range
def fetch_data_for_date_range(start_date, end_date):
    date_list = generate_date_list(start_date, end_date)
    
    for date in date_list:
        # Check if data for this date is in the Redis cache
        cached_data = redis_client.get(date)
        if cached_data:
            print(f"Loading data from cache...")
            events = pickle.loads(cached_data)
            all_events.extend(events)
            continue  # Skip to the next date
        
        # Generate the URL for the current date
        url = f"https://www.forexfactory.com/calendar?day={date}"
        
        # Navigate to the page
        driver.get(url)
        
        # Wait for the page to fully load
        time.sleep(1)
        
        # Locate the table or section containing the calendar events
        try:
            calendar_table = driver.find_element(By.CSS_SELECTOR, 'table.calendar__table')
        except Exception as e:
            print(f"Error loading data for {date}: {e}")
            continue
        
        # Initialize variable to keep track of the current time
        current_time = None
        formatted_date = format_date_for_table(date)  # Format the date for the table
        events = []  # Store events for this date
        
        # Loop through each row in the table
        for row in calendar_table.find_elements(By.CSS_SELECTOR, 'tr.calendar__row'):
            # Extract the time from the td with class calendar__time
            try:
                time_elem = row.find_element(By.CSS_SELECTOR, 'td.calendar__time > div > span')
                current_time = time_elem.text.strip() if time_elem.text.strip() else current_time
            except:
                current_time = current_time  # Keep the last known time

            # Extract the currency from the td with class calendar__currency
            try:
                currency_elem = row.find_element(By.CSS_SELECTOR, 'td.calendar__currency > span')
                currency = currency_elem.text.strip()
            except:
                currency = ""

            # Extract the impact title from the td with class calendar__impact
            try:
                impact_elem = row.find_element(By.CSS_SELECTOR, 'td.calendar__impact > span')
                impact = impact_elem.get_attribute('title').strip()
            except:
                impact = ""

            # Extract the event name from the td with class calendar__event
            try:
                event_elem = row.find_element(By.CSS_SELECTOR, 'td.calendar__event > div > span')
                event_name = event_elem.text.strip()
            except:
                event_name = ""

            # Extract the actual value from the td with class calendar__actual
            try:
                actual_elem = row.find_element(By.CSS_SELECTOR, 'td.calendar__actual > span')
                actual = actual_elem.text.strip()
            except:
                actual = ""

            # Extract the forecast value from the td with class calendar__forecast
            try:
                forecast_elem = row.find_element(By.CSS_SELECTOR, 'td.calendar__forecast > span')
                forecast = forecast_elem.text.strip()
            except:
                forecast = ""

            # Extract the previous value from the td with class calendar__previous
            try:
                previous_elem = row.find_element(By.CSS_SELECTOR, 'td.calendar__previous > span')
                previous = previous_elem.text.strip()
            except:
                previous = ""

            # Apply filters and add to the list if matching criteria
            if currency in desired_currencies and impact in desired_impacts:
                event_data = [formatted_date, current_time, currency, impact, event_name, actual, forecast, previous]
                events.append(event_data)
                all_events.append(event_data)
        
        # Cache the events for this date in Redis
        if events:
            redis_client.set(date, pickle.dumps(events))

# Close the browser
def close_driver():
    driver.quit()

# Print the table
def print_collected_data():
    df_events = pd.DataFrame(all_events, columns=["Date", "Time", "Currency", "Impact", "Event", "Actual", "Forecast", "Previous"])
    print(tabulate(df_events, headers="keys", tablefmt="grid"))

# Main function
def collect_forex_data(start_date, end_date):
    fetch_data_for_date_range(start_date, end_date)
    close_driver()
    print_collected_data()

# Execute
collect_forex_data(start_date, end_date)

+----+------------------------+--------+------------+------------------------+--------------------------------+----------+------------+------------+
|    | Date                   | Time   | Currency   | Impact                 | Event                          | Actual   | Forecast   | Previous   |
+====+========================+========+============+========================+================================+==========+============+============+
|  0 | Monday, 01 February    | 4:00pm | USD        | High Impact Expected   | ISM Manufacturing PMI          | 58.7     | 60.0       | 60.7       |
+----+------------------------+--------+------------+------------------------+--------------------------------+----------+------------+------------+
|  1 | Wednesday, 03 February | 2:15pm | USD        | Medium Impact Expected | ADP Non-Farm Employment Change | 174K     | 48K        | -78K       |
+----+------------------------+--------+------------+------------------------+----------------------------

In [ ]:
# Initialize the Market object
market = Market(market_data_generator=MarketDataGenerator())

# Timeframe choice
timeframe_choice = 'tick'  # Dukascopy supports only tick data

# Function to fetch and analyze data with caching
def fetch_and_analyze(timeframe_choice, start_date, end_date, fetch_one_month_prior=False):
    if fetch_one_month_prior:
        start_date = end_date - timedelta(days=30)

    start_date_str = start_date.strftime('%d %b %Y')
    end_date_str = end_date.strftime('%d %b %Y')
    
    # Fetch data from Dukascopy with caching
    data_frames = []
    for pair in forex_pairs:
        print(f"Fetching data for {pair} ({start_date_str} to {end_date_str})...")
        
        md_request = MarketDataRequest(
            start_date=start_date,
            finish_date=end_date,
            category='fx',
            fields=['bid', 'ask'],
            freq=timeframe_choice,
            data_source='dukascopy',
            tickers=[pair],
            cache_algo='cache_algo_return',  # Use cache algorithm for fetching and saving data
            cut='NYC'  # Adjust time cut to match trading hours
        )
        
        try:
            # Fetch data and cache it
            df = market.fetch_market(md_request)
            
            if df is not None and f'{pair}.bid' in df.columns and f'{pair}.ask' in df.columns:
                df['price'] = (df[f'{pair}.bid'] + df[f'{pair}.ask']) / 2  # Mid price
                df = df[['price']].resample(new_timeframe).mean()
            else:
                print(f"Warning: No valid data for {pair}")
                continue
            
            data_frames.append(df)

            # Retrieve from cache
            cached_df = market.fetch_market(md_request)
            if cached_df is not None:
                print(f"Data for {pair} retrieved from cache ({start_date_str} to {end_date_str})")

        except Exception as e:
            print(f"Error fetching data for {pair}: {e}")
            continue
    
    # Combine all forex pairs into a single DataFrame
    if data_frames:
        combined_df = pd.concat(data_frames, axis=1)
        combined_df.columns = forex_pairs
        combined_df.dropna(inplace=True)
        
        # Calculate correlation matrix
        correlation_matrix = combined_df.corr()
        
        # Plot correlation matrix
        plt.figure(figsize=(10, 8))
        mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
        sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='RdYlGn', vmin=-1, vmax=1, square=True)
        plt.title(f'Correlation Matrix\n{start_date_str} to {end_date_str} ({new_timeframe} timeframe)')
        plt.show()
    else:
        print("No valid data found for the selected forex pairs.")

# Execute with the flag for extended data range
fetch_and_analyze(timeframe_choice, start_date, end_date, fetch_one_month_prior=True)

Fetching data for AUDUSD (06 Jan 2021 to 05 Feb 2021)...
2024-08-26 11:23:16,629 - findatapy.market.ioengine - INFO - Cache not existent for MarketDataRequest_1012__abstract_curve_key-None__arcticdb_dict-{'prune_previous_versions': False, 'write_style': 'write', 'force_create_library': False, 'allow_on_disk_filter': True, 'query_builder': None}__as_of-None__base_depos_currencies-EUR_GBP_AUD_NZD_USD_CAD_CHF_NOK_SEK_JPY__base_depos_tenor-ON_TN_SN_1W_2W_3W_1M_2M_3M_4M_6M_9M_1Y_2Y_3Y_5Y__category-fx__category_key-backtest_fx_dukascopy_tick_NYC_AUDUSD__cut-NYC__data_engine-None__data_source-dukascopy__environment-backtest__expiry_date-NaT__fields-bid_ask__finish_date-2021-02-05 00:00:00__freeform_md_request-{}__freq-tick__freq_mult-1__fx_forwards_tenor-ON_TN_SN_1W_2W_3W_1M_2M_3M_4M_6M_9M_1Y_2Y_3Y_5Y__fx_vol_part-V_25R_10R_25B_10B__fx_vol_tenor-ON_1W_2W_3W_1M_2M_3M_4M_6M_9M_1Y_2Y_3Y_5Y__gran_freq-None__list_threads-1__old_tickers-AUDUSD__pretransformation-None__push_to_cache-True__resample-N